In [1]:
# =========================
# A2 Pairwise (4C2) Full Code
# - Unseen Activity + Unseen Subject
# - For each activity pair (A->B):
#     For each test_subject:
#       Train on activity A using subjects except test_subject (windowed)
#       Test on activity B using ONLY test_subject (windowing inference)
#       k_hat / entropy: full-trial 1 forward for 기록
# - 4C2=6 pairs, BIDIRECTIONAL=True -> 총 12 pair blocks
# - 각 block 안에는 subject1~10 (LOSO) 결과 10줄
# - 결과는 마지막에 한 번만 출력
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from itertools import combinations


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    """
    label_config: list of (subj, act_id, gt_count)
    return: list of dict trials
    """
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score (trial-wise)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,              # (T,C)
                'count': float(gt_count),      # trial total count
                'meta': f"{subj}_{act_name}",
                'subj': subj,
                'act_id': act_id,
            })
        else:
            # missing -> skip
            pass

    return trial_list


# ---------------------------------------------------------------------
# 2.5) Windowing
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=8.0, stride_sec=4.0, drop_last=True):
    """
    TRAIN 전용: trial -> sliding windows
    window 라벨은 trial 평균 rate로부터 생성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]  # (T,C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur  # reps/s

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    TEST 전용: trial -> windows inference -> window rate 평균 -> total count
    x_np: (T,C) numpy (정규화된 상태)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    # short trial -> 1회 forward
    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)
    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N,C,win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count)


# ---------------------------------------------------------------------
# 2.8) Dataset / Collate
# ---------------------------------------------------------------------
class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B,D,T)
        z = z.transpose(1, 2)      # (B,T,D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B,D,T)
        x_hat = self.net(zt)       # (B,C,T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, _ = self.rate_head(z, tau=tau)
        micro_rate_t = amp_t             # (B,T)

        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,)

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6) # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "phase_p": phase_p,          # (B,T,K)
            "rep_rate_t": rep_rate_t,    # (B,T)
            "k_hat": k_hat,              # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)              # (B,1,T)
    se = (x_hat - x) ** 2                    # (B,C,T)
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)  # (B,T)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)
    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean()


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)  # (T,K)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()
        rate_hat, _, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()


# ---------------------------------------------------------------------
# 6) Main (4C2 Pairwise A2)
# ---------------------------------------------------------------------
def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",
        "fs": 50,
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,

        # windowing
        "win_sec": 8.0,
        "stride_sec": 4.0,
        "drop_last": True,

        # model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # loss
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.0075,

        "tau": 1.0,

        "BIDIRECTIONAL": True,

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],
        "TARGET_ACTIVITIES_MAP": {
            6: 'Waist bends forward',
            7: 'Frontal elevation of arms',
            8: 'Knees bending',
            12: 'Jump front & back'
        },

        # ✅ 모든 activity에서 동일 feature list (입력차원 고정)
        "ACT_FEATURE_MAP": {
            6: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'],
            7: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'],
            8: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'],
            12: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                 'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                 'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                 'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                 'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'],
        },

        # COUNT_TABLE[act_id][subject] = GT count
        "COUNT_TABLE": {
            6: {
                "subject1": 21, "subject2": 19, "subject3": 21, "subject4": 20, "subject5": 20,
                "subject6": 20, "subject7": 20, "subject8": 21, "subject9": 21, "subject10": 20,
            },
            7: {
                "subject1": 20, "subject2": 20, "subject3": 20, "subject4": 20, "subject5": 20,
                "subject6": 20, "subject7": 20, "subject8": 19, "subject9": 19, "subject10": 20,
            },
            8: {
                "subject1": 20, "subject2": 21, "subject3": 21, "subject4": 19, "subject5": 20,
                "subject6": 20, "subject7": 21, "subject8": 21, "subject9": 21, "subject10": 12,
            },
            12: {
                "subject1": 20, "subject2": 22, "subject3": 21, "subject4": 21, "subject5": 20,
                "subject6": 21, "subject7": 19, "subject8": 20, "subject9": 20, "subject10": 20,
            },
        }
    }

    # -----------------------------
    # sanity: feature 동일성 체크
    # -----------------------------
    feats_ref = None
    for act_id, feats in CONFIG["ACT_FEATURE_MAP"].items():
        if feats_ref is None:
            feats_ref = tuple(feats)
        elif tuple(feats) != feats_ref:
            raise ValueError(f"[ERROR] ACT_FEATURE_MAP must be identical across activities. mismatch at act_id={act_id}")

    set_strict_seed(CONFIG["seed"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    full_data = load_mhealth_dataset(CONFIG["data_dir"], CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["COLUMN_NAMES"])
    if not full_data:
        print("[ERROR] dataset load failed")
        return

    subjects = [f"subject{i}" for i in range(1, 11)]
    act_ids = sorted(list(CONFIG["TARGET_ACTIVITIES_MAP"].keys()))  # 4 activities
    pairs = list(combinations(act_ids, 2))  # 4C2 = 6

    # 실험 방향 리스트(6 or 12)
    exp_dirs = []
    for a, b in pairs:
        exp_dirs.append((a, b))
        if CONFIG["BIDIRECTIONAL"]:
            exp_dirs.append((b, a))

    def gt_count(act_id, subj):
        if act_id not in CONFIG["COUNT_TABLE"]:
            return None
        return CONFIG["COUNT_TABLE"][act_id].get(subj, None)

    final_blocks = []
    exp_idx = 0

    # -----------------------------
    # A2 loop: (actA->actB) x LOSO(subject)
    # -----------------------------
    for train_act, test_act in exp_dirs:
        exp_idx += 1
        train_name = CONFIG["TARGET_ACTIVITIES_MAP"][train_act]
        test_name  = CONFIG["TARGET_ACTIVITIES_MAP"][test_act]

        block_lines = []
        abs_errs = []

        for test_subj in subjects:
            # -------------------------
            # build labels (A2 rule)
            # Train: train_act with subjects != test_subj
            # Test : test_act  with subject == test_subj
            # -------------------------
            train_labels = []
            for s in subjects:
                if s == test_subj:
                    continue
                gtc = gt_count(train_act, s)
                if gtc is None:
                    continue
                train_labels.append((s, train_act, gtc))

            test_labels = []
            gtc_test = gt_count(test_act, test_subj)
            if gtc_test is not None:
                test_labels.append((test_subj, test_act, gtc_test))

            # missing labels -> write SKIP line
            if len(train_labels) == 0 or len(test_labels) == 0:
                block_lines.append(f"{test_subj} | SKIP (missing COUNT_TABLE or data)")
                continue

            # seed per fold
            set_strict_seed(CONFIG["seed"])

            # trial-level load
            train_trials = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
            test_trials  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

            if len(train_trials) == 0 or len(test_trials) == 0:
                block_lines.append(f"{test_subj} | SKIP (missing log segment)")
                continue

            # train windows
            train_windows = trial_list_to_windows(
                train_trials,
                fs=CONFIG["fs"],
                win_sec=CONFIG["win_sec"],
                stride_sec=CONFIG["stride_sec"],
                drop_last=CONFIG["drop_last"]
            )
            if len(train_windows) == 0:
                block_lines.append(f"{test_subj} | SKIP (no windows)")
                continue

            g = torch.Generator()
            g.manual_seed(CONFIG["seed"])
            train_loader = DataLoader(
                TrialDataset(train_windows),
                batch_size=CONFIG["batch_size"],
                shuffle=True,
                collate_fn=collate_variable_length,
                generator=g,
                num_workers=0
            )

            # model
            input_ch = train_windows[0]["data"].shape[1]
            model = KAutoCountModel(
                input_ch=input_ch,
                hidden_dim=CONFIG["hidden_dim"],
                latent_dim=CONFIG["latent_dim"],
                K_max=CONFIG["K_max"]
            ).to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

            # train (silent)
            for _ in range(CONFIG["epochs"]):
                train_one_epoch(model, train_loader, optimizer, CONFIG, device)
                scheduler.step()

            model.eval()

            # test_trials has exactly 1 item (test_subj, test_act)
            item = test_trials[0]
            x_np = item["data"]
            gt = float(item["count"])

            # windowing inference for count
            pred = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["win_sec"],
                stride_sec=CONFIG["stride_sec"],
                device=device,
                tau=CONFIG.get("tau", 1.0),
                batch_size=CONFIG.get("batch_size", 64)
            )
            diff = pred - gt
            abs_errs.append(abs(diff))

            # full-trial forward for k_hat & entropy
            x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
            with torch.no_grad():
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG.get("tau", 1.0))

            phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()  # (T,K)
            k_hat = float(aux["k_hat"].item())
            ent = compute_phase_entropy_mean(phase_p)

            line = (f"{test_subj} | GT={gt:.0f} | Pred(win)={pred:.2f} | Diff={diff:+.2f} | "
                    f"k_hat(full)={k_hat:.2f} | phase_entropy(full)={ent:.3f}")
            block_lines.append(line)

        mae = float(np.mean(abs_errs)) if len(abs_errs) > 0 else float("nan")

        final_blocks.append({
            "idx": exp_idx,
            "title": f"{train_name} -> {test_name}",
            "lines": block_lines,
            "mae": mae
        })

    # -----------------------------
    # print ONCE at the end
    # -----------------------------
    print("\n" + "=" * 110)
    print("A2 Pairwise Activity+Subject Transfer Results (4C2 pairs, LOSO-subject inside each pair)")
    print("=" * 110)

    if len(final_blocks) == 0:
        print("[No results] COUNT_TABLE이 충분히 채워졌는지 확인해줘.")
        print("=" * 110)
        return

    for b in final_blocks:
        print(f"\n{b['idx']}. {b['title']}  |  MAE={b['mae']:.3f}")
        for ln in b["lines"]:
            print(ln)

    maes = [b["mae"] for b in final_blocks if np.isfinite(b["mae"])]
    if len(maes) > 0:
        print("\n" + "-" * 110)
        print(f"Overall Avg MAE={float(np.mean(maes)):.3f} | Std={float(np.std(maes)):.3f} | #pair_blocks={len(maes)}")
        print("-" * 110)

    print("=" * 110)


if __name__ == "__main__":
    main()



A2 Pairwise Activity+Subject Transfer Results (4C2 pairs, LOSO-subject inside each pair)

1. Waist bends forward -> Frontal elevation of arms  |  MAE=6.008
subject1 | GT=20 | Pred(win)=27.00 | Diff=+7.00 | k_hat(full)=1.02 | phase_entropy(full)=0.043
subject2 | GT=20 | Pred(win)=29.74 | Diff=+9.74 | k_hat(full)=1.06 | phase_entropy(full)=0.123
subject3 | GT=20 | Pred(win)=22.93 | Diff=+2.93 | k_hat(full)=1.03 | phase_entropy(full)=0.069
subject4 | GT=20 | Pred(win)=28.34 | Diff=+8.34 | k_hat(full)=1.02 | phase_entropy(full)=0.047
subject5 | GT=20 | Pred(win)=26.35 | Diff=+6.35 | k_hat(full)=1.02 | phase_entropy(full)=0.051
subject6 | GT=20 | Pred(win)=14.95 | Diff=-5.05 | k_hat(full)=1.01 | phase_entropy(full)=0.026
subject7 | GT=20 | Pred(win)=21.33 | Diff=+1.33 | k_hat(full)=1.02 | phase_entropy(full)=0.042
subject8 | GT=19 | Pred(win)=33.73 | Diff=+14.73 | k_hat(full)=1.01 | phase_entropy(full)=0.033
subject9 | GT=19 | Pred(win)=23.55 | Diff=+4.55 | k_hat(full)=1.01 | phase_entropy

In [2]:
# =========================
# A2 Pairwise (4C2) Full Code
# - Unseen Activity + Unseen Subject
# - For each activity pair (A->B):
#     For each test_subject:
#       Train on activity A using subjects except test_subject (windowed)
#       Test on activity B using ONLY test_subject (windowing inference)
#       k_hat / entropy: full-trial 1 forward for 기록
# - 4C2=6 pairs, BIDIRECTIONAL=True -> 총 12 pair blocks
# - 각 block 안에는 subject1~10 (LOSO) 결과 10줄
# - 결과는 마지막에 한 번만 출력
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from itertools import combinations


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    """
    label_config: list of (subj, act_id, gt_count)
    return: list of dict trials
    """
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score (trial-wise)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,              # (T,C)
                'count': float(gt_count),      # trial total count
                'meta': f"{subj}_{act_name}",
                'subj': subj,
                'act_id': act_id,
            })
        else:
            # missing -> skip
            pass

    return trial_list


# ---------------------------------------------------------------------
# 2.5) Windowing
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=8.0, stride_sec=4.0, drop_last=True):
    """
    TRAIN 전용: trial -> sliding windows
    window 라벨은 trial 평균 rate로부터 생성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]  # (T,C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur  # reps/s

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    TEST 전용: trial -> windows inference -> window rate 평균 -> total count
    x_np: (T,C) numpy (정규화된 상태)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    # short trial -> 1회 forward
    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)
    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N,C,win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count)


# ---------------------------------------------------------------------
# 2.8) Dataset / Collate
# ---------------------------------------------------------------------
class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B,D,T)
        z = z.transpose(1, 2)      # (B,T,D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B,D,T)
        x_hat = self.net(zt)       # (B,C,T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, _ = self.rate_head(z, tau=tau)
        micro_rate_t = amp_t             # (B,T)

        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,)

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6) # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "phase_p": phase_p,          # (B,T,K)
            "rep_rate_t": rep_rate_t,    # (B,T)
            "k_hat": k_hat,              # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)              # (B,1,T)
    se = (x_hat - x) ** 2                    # (B,C,T)
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)  # (B,T)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)
    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean()


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)  # (T,K)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()
        rate_hat, _, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()


# ---------------------------------------------------------------------
# 6) Main (4C2 Pairwise A2)
# ---------------------------------------------------------------------
def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",
        "fs": 50,
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,

        # windowing
        "win_sec": 8.0,
        "stride_sec": 4.0,
        "drop_last": True,

        # model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # loss
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.0075,

        "tau": 1.0,

        "BIDIRECTIONAL": True,

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],
        "TARGET_ACTIVITIES_MAP": {
            4: 'Walking',
            10: 'Jogging',
            11: 'Running',
        },

        # ✅ 모든 activity에서 동일 feature list여야 pairwise 전이가 깔끔함(입력차원 고정)
        "ACT_FEATURE_MAP": {
            4: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'],
            10: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'],
            11: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'],
        },

        # COUNT_TABLE[act_id][subject] = GT count
        "COUNT_TABLE": {
            4: {
                "subject1": 113, "subject2": 99, "subject3": 104, "subject4": 112, "subject5": 109,
                "subject6": 111, "subject7": 106, "subject8": 95, "subject9": 111, "subject10": 102,
            },
            10: {
                "subject1": 157, "subject2": 161, "subject3": 154, "subject4": 154, "subject5": 160,
                "subject6": 156, "subject7": 153, "subject8": 160, "subject9": 166, "subject10": 156,
            },
            11: {
                "subject1": 165, "subject2": 158, "subject3": 174, "subject4": 163, "subject5": 157,
                "subject6": 172, "subject7": 149, "subject8": 166, "subject9": 174, "subject10": 172,
            },
        }
    }

    # -----------------------------
    # sanity: feature 동일성 체크
    # -----------------------------
    feats_ref = None
    for act_id, feats in CONFIG["ACT_FEATURE_MAP"].items():
        if feats_ref is None:
            feats_ref = tuple(feats)
        elif tuple(feats) != feats_ref:
            raise ValueError(f"[ERROR] ACT_FEATURE_MAP must be identical across activities. mismatch at act_id={act_id}")

    set_strict_seed(CONFIG["seed"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    full_data = load_mhealth_dataset(CONFIG["data_dir"], CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["COLUMN_NAMES"])
    if not full_data:
        print("[ERROR] dataset load failed")
        return

    subjects = [f"subject{i}" for i in range(1, 11)]
    act_ids = sorted(list(CONFIG["TARGET_ACTIVITIES_MAP"].keys()))  # 4 activities
    pairs = list(combinations(act_ids, 2))  # 4C2 = 6

    # 실험 방향 리스트(6 or 12)
    exp_dirs = []
    for a, b in pairs:
        exp_dirs.append((a, b))
        if CONFIG["BIDIRECTIONAL"]:
            exp_dirs.append((b, a))

    def gt_count(act_id, subj):
        if act_id not in CONFIG["COUNT_TABLE"]:
            return None
        return CONFIG["COUNT_TABLE"][act_id].get(subj, None)

    final_blocks = []
    exp_idx = 0

    # -----------------------------
    # A2 loop: (actA->actB) x LOSO(subject)
    # -----------------------------
    for train_act, test_act in exp_dirs:
        exp_idx += 1
        train_name = CONFIG["TARGET_ACTIVITIES_MAP"][train_act]
        test_name  = CONFIG["TARGET_ACTIVITIES_MAP"][test_act]

        block_lines = []
        abs_errs = []

        for test_subj in subjects:
            # -------------------------
            # build labels (A2 rule)
            # Train: train_act with subjects != test_subj
            # Test : test_act  with subject == test_subj
            # -------------------------
            train_labels = []
            for s in subjects:
                if s == test_subj:
                    continue
                gtc = gt_count(train_act, s)
                if gtc is None:
                    continue
                train_labels.append((s, train_act, gtc))

            test_labels = []
            gtc_test = gt_count(test_act, test_subj)
            if gtc_test is not None:
                test_labels.append((test_subj, test_act, gtc_test))

            # missing labels -> write SKIP line
            if len(train_labels) == 0 or len(test_labels) == 0:
                block_lines.append(f"{test_subj} | SKIP (missing COUNT_TABLE or data)")
                continue

            # seed per fold
            set_strict_seed(CONFIG["seed"])

            # trial-level load
            train_trials = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
            test_trials  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

            if len(train_trials) == 0 or len(test_trials) == 0:
                block_lines.append(f"{test_subj} | SKIP (missing log segment)")
                continue

            # train windows
            train_windows = trial_list_to_windows(
                train_trials,
                fs=CONFIG["fs"],
                win_sec=CONFIG["win_sec"],
                stride_sec=CONFIG["stride_sec"],
                drop_last=CONFIG["drop_last"]
            )
            if len(train_windows) == 0:
                block_lines.append(f"{test_subj} | SKIP (no windows)")
                continue

            g = torch.Generator()
            g.manual_seed(CONFIG["seed"])
            train_loader = DataLoader(
                TrialDataset(train_windows),
                batch_size=CONFIG["batch_size"],
                shuffle=True,
                collate_fn=collate_variable_length,
                generator=g,
                num_workers=0
            )

            # model
            input_ch = train_windows[0]["data"].shape[1]
            model = KAutoCountModel(
                input_ch=input_ch,
                hidden_dim=CONFIG["hidden_dim"],
                latent_dim=CONFIG["latent_dim"],
                K_max=CONFIG["K_max"]
            ).to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

            # train (silent)
            for _ in range(CONFIG["epochs"]):
                train_one_epoch(model, train_loader, optimizer, CONFIG, device)
                scheduler.step()

            model.eval()

            # test_trials has exactly 1 item (test_subj, test_act)
            item = test_trials[0]
            x_np = item["data"]
            gt = float(item["count"])

            # windowing inference for count
            pred = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["win_sec"],
                stride_sec=CONFIG["stride_sec"],
                device=device,
                tau=CONFIG.get("tau", 1.0),
                batch_size=CONFIG.get("batch_size", 64)
            )
            diff = pred - gt
            abs_errs.append(abs(diff))

            # full-trial forward for k_hat & entropy
            x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
            with torch.no_grad():
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG.get("tau", 1.0))

            phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()  # (T,K)
            k_hat = float(aux["k_hat"].item())
            ent = compute_phase_entropy_mean(phase_p)

            line = (f"{test_subj} | GT={gt:.0f} | Pred(win)={pred:.2f} | Diff={diff:+.2f} | "
                    f"k_hat(full)={k_hat:.2f} | phase_entropy(full)={ent:.3f}")
            block_lines.append(line)

        mae = float(np.mean(abs_errs)) if len(abs_errs) > 0 else float("nan")

        final_blocks.append({
            "idx": exp_idx,
            "title": f"{train_name} -> {test_name}",
            "lines": block_lines,
            "mae": mae
        })

    # -----------------------------
    # print ONCE at the end
    # -----------------------------
    print("\n" + "=" * 110)
    print("A2 Pairwise Activity+Subject Transfer Results (4C2 pairs, LOSO-subject inside each pair)")
    print("=" * 110)

    if len(final_blocks) == 0:
        print("[No results] COUNT_TABLE이 충분히 채워졌는지 확인해줘.")
        print("=" * 110)
        return

    for b in final_blocks:
        print(f"\n{b['idx']}. {b['title']}  |  MAE={b['mae']:.3f}")
        for ln in b["lines"]:
            print(ln)

    maes = [b["mae"] for b in final_blocks if np.isfinite(b["mae"])]
    if len(maes) > 0:
        print("\n" + "-" * 110)
        print(f"Overall Avg MAE={float(np.mean(maes)):.3f} | Std={float(np.std(maes)):.3f} | #pair_blocks={len(maes)}")
        print("-" * 110)

    print("=" * 110)


if __name__ == "__main__":
    main()



A2 Pairwise Activity+Subject Transfer Results (4C2 pairs, LOSO-subject inside each pair)

1. Walking -> Jogging  |  MAE=52.840
subject1 | GT=157 | Pred(win)=102.21 | Diff=-54.79 | k_hat(full)=2.31 | phase_entropy(full)=0.635
subject2 | GT=161 | Pred(win)=96.42 | Diff=-64.58 | k_hat(full)=2.51 | phase_entropy(full)=0.716
subject3 | GT=154 | Pred(win)=96.80 | Diff=-57.20 | k_hat(full)=2.32 | phase_entropy(full)=0.742
subject4 | GT=154 | Pred(win)=102.68 | Diff=-51.32 | k_hat(full)=2.25 | phase_entropy(full)=0.701
subject5 | GT=160 | Pred(win)=98.36 | Diff=-61.64 | k_hat(full)=2.47 | phase_entropy(full)=0.688
subject6 | GT=156 | Pred(win)=144.03 | Diff=-11.97 | k_hat(full)=1.71 | phase_entropy(full)=0.623
subject7 | GT=153 | Pred(win)=102.50 | Diff=-50.50 | k_hat(full)=2.35 | phase_entropy(full)=0.709
subject8 | GT=160 | Pred(win)=93.09 | Diff=-66.91 | k_hat(full)=2.33 | phase_entropy(full)=0.673
subject9 | GT=166 | Pred(win)=116.04 | Diff=-49.96 | k_hat(full)=2.01 | phase_entropy(full)=